## Chapter 3 - Exploring Classical Machine Learning

This chapter dives into classical machine learning techniques using Scikit-learn, focusing on regression, classification, dimensionality reduction, and clustering. We begin with a linear regression example to predict superhero height from weight, followed by classification models (Logistic Regression and Decision Trees) to categorize superheroes by race based on their powers. The notebook then explores dimensionality reduction using PCA to create a "Power Score" and identifies optimal components for capturing variance. Clustering is applied to group superheroes based on their PCA scores and alignment, followed by an analysis of cluster characteristics and cosine similarity to find similar and dissimilar pairs. Finally, the chapter prepares for supervised modeling by screening potential targets, building features, and establishing a baseline with a Gradient Boosting classifier. It then fine-tunes the model, visualizes the results with a confusion matrix, and uses SHAP values to explain model predictions.

**Note:** Run the following cell to define constants related to datasets

In [ ]:
# Base GitHub repository URL
BASE_URL = "https://buildyourownai.github.io/code/datasets/"

# Dataset file names
POWERS_FILE       = "superheroes_powers.csv"
INFO_POWERS_FILE  = "superheroes_info_powers.csv"
INFO_POWERS2_FILE = "superheroes_info_powers2.csv"
INFO_POWERS3_FILE = "superheroes_info_powers3.csv"
PLOTS_FILE        = "superheroes_story_plots.csv"

# Construct full dataset URLs
SUPERHEROES_POWERS_URL = f"{BASE_URL}{POWERS_FILE}"
SUPERHEROES_INFO_POWERS_URL = f"{BASE_URL}{INFO_POWERS_FILE}"
SUPERHEROES_INFO_POWERS2_URL = f"{BASE_URL}{INFO_POWERS2_FILE}"
SUPERHEROES_INFO_POWERS3_URL = f"{BASE_URL}{INFO_POWERS2_FILE}"
SUPERHEROES_INFO_PLOTS_URL = f"{BASE_URL}{PLOTS_FILE}"

### Listing 3-1: Linear Regression of Superhero Height and Weight
Performs linear regression to predict height from weight, using filtered superhero data and evaluating with the coefficient, intercept, and mean squared error. Then Plots the actual versus predicted heights, visualizing the linear regression model’s performance and highlighting how well the model fits the superhero data.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Helper (Colab only): tidy regression report
def show_regression_report(model, mse, r2, feature_names=None):
    coefs = getattr(model, "coef_", [])
    coefs = coefs.ravel() if hasattr(coefs, "ravel") else coefs
    if feature_names is None:
        feature_names = [f"x{i}" for i in range(len(coefs))]
    print("Regression Report")
    print("-----------------")
    for name, c in zip(feature_names, coefs):
        print(f"  coef[{name:<8}] : {c:>10.4f}")
    if hasattr(model, "intercept_"):
        print(f"  intercept   : {model.intercept_:>10.2f}")
    print(f"  MSE         : {mse:>10.2f}")
    print(f"  R2          : {r2:>10.3f}")

def plot_regression(X_test, y_test, y_pred, title=None,
                    xlab="Weight", ylab="Height"):
    import numpy as np
    import matplotlib.pyplot as plt

    # Sort by X so the prediction line draws correctly
    order = np.argsort(X_test.ravel())
    Xs = X_test.ravel()[order]
    ys = y_test.ravel()[order]
    yhat = y_pred.ravel()[order]

    plt.scatter(Xs, ys, color="blue", label="Actual Heights")
    plt.plot(Xs, yhat, color="red", label="Predicted Heights")
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.title(title or "Linear Regression: Height vs. Weight (Human & Cyborg)")
    plt.legend(loc="upper left")
    plt.show()

# --------- Main Control Flow

# Step 0: Load
df = pd.read_csv(SUPERHEROES_INFO_POWERS_URL)

# Step 1: Prep — filter, drop NAs, trim extremes
df = df[df['Species'].isin(['Human', 'Cyborg'])].dropna(
    subset=['Height', 'Weight']
)
df = df[df['Weight'].between(30, 400) & df['Height'].between(100, 350)]

# Step 2: Split independent features (X) and dependent target (y)
X = df[['Weight']].values
y = df['Height'].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 3: Train
model = LinearRegression().fit(X_train, y_train)

# Step 4: Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = model.score(X_test, y_test)

# Step 5: Create regression report and plot results
show_regression_report(model, mse, r2, feature_names=["Weight"])
plot_regression(
    X_test, y_test, y_pred,
    title="Linear Regression: Height vs. Weight (Human & Cyborg)"
)

### Listing 3-2: Training Classification Models for Superhero Races
Trains logistic regression and decision tree models to classify superheroes into races (Human, Mutant, Cyborg) using powers data, then outputs accuracy and performance metrics. Then Visualizes the decision tree using a simplified, color-coded representation to help understand how the model makes predictions based on superhero attributes.

In [ ]:
# REQUIRES: SUPERHEROES_POWERS_URL, SUPERHEROES_INFO_POWERS_URL

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Helpers for Colab only; omit from book listing
def load_and_prepare(powers_url, info_url):
    """
    Returns:
      X: all power flags as numeric features (independent vars)
      y: Species labels mapped to integers (dependent var)
    """
    df_p = pd.read_csv(powers_url)
    df_i = pd.read_csv(info_url)[['name', 'Species']]
    df = df_p.merge(df_i, left_on='hero_names', right_on='name')
    df = df[df['Species'].isin(['Human', 'Mutant', 'Cyborg'])].dropna()
    # Labels y: Species -> int
    y = df['Species'].map({'Human': 0, 'Mutant': 1, 'Cyborg': 2}).values
    # Features X: all power columns only
    X = df.drop(columns=['hero_names', 'name', 'Species']).astype(int).values
    return X, y

def train_and_eval(model, X_train, X_test, y_train, y_test):
    """
    Fit on training data, predict on test data, return accuracy + report.
    """
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    acc = accuracy_score(y_test, y_hat)
    rep = classification_report(y_test, y_hat, zero_division=0)
    return acc, rep

def print_report(title, acc, rep):
    print(f"{title} Accuracy: {acc:.3f}")
    print(f"{title} Report:\n{rep}")

# Step 0: Load and prepare data  (X = powers, y = Species)
X, y = load_and_prepare(SUPERHEROES_POWERS_URL,
                        SUPERHEROES_INFO_POWERS_URL)

# Step 1: Train/test split (hold out 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 2: Define models
#   LogisticRegression: fast baseline
#   DecisionTree: interpretable, non-linear splits
log_model = LogisticRegression(max_iter=1000)
tree_model = DecisionTreeClassifier(random_state=42, max_depth=5)

# Step 3: Train and evaluate
log_acc, log_rep = train_and_eval(log_model, X_train, X_test,
                                  y_train, y_test)
tree_acc, tree_rep = train_and_eval(tree_model, X_train, X_test,
                                    y_train, y_test)

# Step 4: Show results
print_report("Logistic Regression", log_acc, log_rep)
print_report("Decision Tree", tree_acc, tree_rep)

Run the following code cell to plot the decision tree

In [ ]:
# REQUIRES before running this cell:
#   - Packages: pandas, scikit-learn, matplotlib
#   - Data paths/URLs defined: SUPERHEROES_POWERS_URL, SUPERHEROES_INFO_POWERS_URL

# =========================
# ------ CONSTANTS --------
# =========================
FIG_W, FIG_H        = 42, 22        # big canvas; extra vertical room for lower leaves
TITLE_SIZE          = 48            # headline-large for slides
LABEL_SIZE          = 42            # large but avoids label collisions
LEGEND_SIZE         = 42

LEGEND_SIZE         = 44
LEGEND_BOX_W, LEGEND_BOX_H = 44, 18  # points-ish; tune to taste

EDGE_WIDTH          = 3.0           # darker, thicker edges pop on projectors
EDGE_COLOR          = "k"
NODE_MARKERSIZE     = 9
LEAF_SIZE           = 0.20          # slightly larger class blocks
LEAF_EDGE_COLOR     = "k"
LEAF_EDGE_WIDTH     = 0.8

LABEL_BOX_ALPHA     = 0.90          # more opaque; text stays crisp over lines
LABEL_BOX_EDGE      = "0.5"
LABEL_Y_OFFSET      = 0.045         # a bit more clearance above node
NAME_TRUNC          = 28            # keeps labels short enough across the width
MAX_RULES           = 12            # shows top splits without crowding

CLASSES             = ["Human", "Mutant", "Cyborg"]
COLORS              = ["blue", "red", "yellow"]

# =========================
# ------ HELPERS ----------
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

def keyify(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

def to01(s: pd.Series) -> pd.Series:
    if s.dtype == "bool":
        return s.astype(int)
    if s.dtype == "O":
        return s.astype(str).str.lower().isin(["true","1","yes"]).astype(int)
    return s.fillna(0).astype(int)

# Minimal tree viz with human-readable labels (keeps your original draw order)
def plot_annotated_tree(tree_model, feature_names=None,
                        title="Decision Tree", annotate=True, max_rules=MAX_RULES):
    # prefer names learned by sklearn; else use provided list
    if hasattr(tree_model, "feature_names_in_"):
        names = list(tree_model.feature_names_in_)
    else:
        names = list(feature_names) if feature_names is not None else None

    def short_label(idx: int) -> str:
        if names and 0 <= idx < len(names):
            n = str(names[idx]).strip()
            return (n[:NAME_TRUNC] + ("…" if len(n) > NAME_TRUNC else "")) + "?"
        return f"feat_{idx}?"

    tree = tree_model.tree_
    fig, ax = plt.subplots(figsize=(FIG_W, FIG_H))
    rules, rid = [], 0

    def plot_node(node, depth, x, width):
        nonlocal rid
        left, right = tree.children_left[node], tree.children_right[node]
        if left != right:  # internal
            lx, rx = x - width/2, x + width/2
            nxt = width/2
            ax.plot([x, lx], [depth, depth + 1], color=EDGE_COLOR, linewidth=EDGE_WIDTH)
            ax.plot([x, rx], [depth, depth + 1], color=EDGE_COLOR, linewidth=EDGE_WIDTH)
            if annotate and rid < max_rules:
                feat = int(tree.feature[node])
                rid += 1
                lab = f"{rid}: {short_label(feat)}"
                ax.text(x, depth - LABEL_Y_OFFSET, lab, ha="center", va="top",
                        fontsize=LABEL_SIZE,
                        bbox=dict(facecolor="white", alpha=LABEL_BOX_ALPHA,
                                  edgecolor=LABEL_BOX_EDGE))
                rules.append(lab)
            ax.plot(x, depth, "ko", markersize=NODE_MARKERSIZE)
            plot_node(left, depth + 1, lx, nxt)
            plot_node(right, depth + 1, rx, nxt)
        else:  # leaf
            vals = tree.value[node].ravel()
            pred = int(np.argmax(vals))
            half = LEAF_SIZE / 2.0
            ax.add_patch(plt.Rectangle((x - half, depth - half),
                                       LEAF_SIZE, LEAF_SIZE,
                                       facecolor=COLORS[pred],
                                       edgecolor=LEAF_EDGE_COLOR,
                                       linewidth=LEAF_EDGE_WIDTH))

    plot_node(0, 0, 0.5, 1.0)
    ax.axis("off")
    patches = [mpatches.Patch(color=c, label=l) for c, l in zip(COLORS, CLASSES)]

    leg = ax.legend(
        handles=patches,
        loc="upper right",
        framealpha=0.95,
        fontsize=LEGEND_SIZE,   # legend text size
        handlelength=2.2,       # widens the color swatch
        borderpad=0.6,
        labelspacing=0.6,
        handletextpad=0.6,
    )

    # enlarge the color boxes (works across Matplotlib versions)
    for p in leg.get_patches():
        p.set_width(LEGEND_BOX_W)
        p.set_height(LEGEND_BOX_H)

    ax.set_title(title, fontsize=TITLE_SIZE)
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()

    if annotate and rules:
        print("Annotated split questions (top of tree):")
        for r in rules:
            print(f"  {r}")

# =========================
# ---- MAIN CONTROL -------
# =========================
# Load data
pow_df  = pd.read_csv(SUPERHEROES_POWERS_URL)       # 160+ flags
info_df = pd.read_csv(SUPERHEROES_INFO_POWERS_URL)  # names + Species

# Join on common key
pow_key = "hero_names" if "hero_names" in pow_df.columns else "name"
inf_key = "name" if "name" in info_df.columns else "hero_names"
pow_df["name_key"]  = keyify(pow_df[pow_key])
info_df["name_key"] = keyify(info_df[inf_key])
dfm = pow_df.merge(info_df[["name_key", "Species"]], on="name_key", how="inner")

# Keep target classes
valid = ["Human", "Mutant", "Cyborg"]
dfm = dfm[dfm["Species"].isin(valid)].copy()

# Select power columns (exclude metadata)
exclude = {"name_key", "Species", "name", "hero_names",
           "Height", "Weight", "has_powers_source"}
feat_cols = [c for c in dfm.columns if c not in exclude]

# Features/labels
X_df = dfm[feat_cols].apply(to01)
y_cls = dfm["Species"].map({"Human":0, "Mutant":1, "Cyborg":2}).astype(int)

print(f"features={X_df.shape[1]}  class_counts={y_cls.value_counts().to_dict()}")

# Train/test and fit
X_tr, X_te, y_tr, y_te = train_test_split(X_df, y_cls, test_size=0.2, random_state=42)
tree = DecisionTreeClassifier(random_state=42, max_depth=5)
tree.fit(X_tr, y_tr)

# Plot
plot_annotated_tree(
    tree,
    feature_names=X_df.columns.tolist(),
    title="Decision Tree: Rule Paths for Species Prediction",
    annotate=True,
    max_rules=MAX_RULES
)

### Listing 3-3: Dimensionality Reduction with PCA Power Score
This code uses PCA for dimensionality reduction, enhancing the superheroes_info_powers dataset with a more nuanced Power Score, offering a comprehensive measure of superheroes' abilities.

In [ ]:
# REQUIRES: SUPERHEROES_INFO_POWERS_URL, SUPERHEROES_POWERS_URL

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

MIN_POWER_FLAGS = 2  # require ≥2 powers to count as a valid profile

# ----------------------- Helper: validity mask -----------------------
def valid_power_mask(df: pd.DataFrame, X: pd.DataFrame) -> pd.Series:
    """True for rows with a source and ≥ MIN_POWER_FLAGS set.
       Also require OPR+SDR > 0 when present."""
    has_src = df['has_powers_source'] if 'has_powers_source' in df.columns \
              else pd.Series(1, index=df.index)
    power_count = X.fillna(0).astype(float).sum(axis=1)
    flags_ok = power_count >= MIN_POWER_FLAGS          # exclude 0–1 flags
    if {'OPR','SDR'}.issubset(df.columns):
        opr_sdr_ok = (df['OPR'].fillna(0) + df['SDR'].fillna(0)) > 0
    else:
        opr_sdr_ok = pd.Series(True, index=df.index)
    return (has_src == 1) & flags_ok & opr_sdr_ok

# ----------------------------- Control ------------------------------

# Load data
info = pd.read_csv(SUPERHEROES_INFO_POWERS_URL)
powers = pd.read_csv(SUPERHEROES_POWERS_URL)

# Join on hero name
df = info.merge(powers, left_on='name', right_on='hero_names')

# Build power matrix
non_power = [
    'name','Gender','Species','Height','Publisher','Alignment',
    'Weight','OPR','SDR','hero_names','has_powers_source'
]
X = df.drop(columns=[c for c in non_power if c in df.columns], errors='ignore')

# Mask rows with real evidence (source present, ≥2 powers, OPR+SDR>0)
mask = valid_power_mask(df, X)                          # boolean per row

# Fit scaler on valid rows only (avoid bias from placeholders)
X_fit = X.loc[mask].fillna(0).astype(float)             # numeric, no NaNs
scaler = StandardScaler()                               # create scaler
Xz_fit = scaler.fit_transform(X_fit)                    # standardize valid set

# Fit PCA on valid rows only (teach with 1D PC1; expand later if needed)
pca = PCA(n_components=1)                               # first component only
pc1_fit = pca.fit_transform(Xz_fit).ravel()             # per-hero PC1 (valid)
evr = float(pca.explained_variance_ratio_[0])           # variance share of PC1

# Transform all rows consistently, assign scores only to valid rows
X_all = X.fillna(0).astype(float)                       # same prep for all
pc1_all = pca.transform(scaler.transform(X_all)).ravel()# project everyone

scores = pd.Series(np.nan, index=df.index)              # start as missing
scores.loc[mask] = np.round(pc1_all[mask], 1)           # write valid scores
df['PCA_Power_Score'] = scores                          # keep NaN for 0–1 flags

# Map back to original info table
df_info_powers = info.merge(
    df[['name','PCA_Power_Score']], on='name', how='left'
)

# Quick preview
print('Explained variance (PC1):', round(evr, 3))        # e.g., 0.056
print('Skipped as invalid (<=1 power or OPR+SDR==0):',
      int((~mask).sum()))
print(df_info_powers.head())

### Listing 3-3A: Finding the Optimal Number of PCA Components
This section refines the PCA process by exploring how many components are needed to capture a desired amount of variance in the power data. It allows experimentation with the `N_COMPONENTS` or `VAR_TARGET` constants to determine the best dimensionality reduction for representing the superhero powers effectively.

In [ ]:
"""
PCA reference runner with adjustable components.
- MODE: "k" (keep first N_COMPONENTS) or "var" (hit VAR_TARGET).
- Validity mask excludes rows with weak/incomplete power info.
- Fits scaler+PCA on valid rows, then transforms all rows.
- Reattaches PCs; invalid rows get NaN for all PCA outputs.
"""

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ------------------------------- CONFIG --------------------------------
MODE = "k"          # "k" for fixed number; "var" to target variance
N_COMPONENTS = 80   # used if MODE == "k"
VAR_TARGET  = 0.80  # used if MODE == "var" (e.g., 0.80 for 80%)
MIN_POWER_FLAGS = 2 # require ≥2 power flags to consider a row "valid"

# --------------------------- HELPERS -----------------------------------

def join_on_name(info_url: str, powers_url: str) -> pd.DataFrame:
    info = pd.read_csv(info_url)
    powers = pd.read_csv(powers_url)
    return info.merge(powers, left_on='name', right_on='hero_names')

def build_power_matrix(df: pd.DataFrame) -> pd.DataFrame:
    non_power = [
        'name','Gender','Species','Height','Publisher','Alignment',
        'Weight','OPR','SDR','hero_names','has_powers_source'
    ]
    return df.drop(columns=[c for c in non_power if c in df.columns],
                   errors='ignore')

def valid_power_mask(df: pd.DataFrame, X: pd.DataFrame) -> pd.Series:
    """True for rows with a source and ≥ MIN_POWER_FLAGS set.
       Also require OPR+SDR > 0 when present."""
    has_src = df['has_powers_source'] if 'has_powers_source' in df.columns \
              else pd.Series(1, index=df.index)
    power_count = X.fillna(0).astype(float).sum(axis=1)
    flags_ok = power_count >= MIN_POWER_FLAGS
    if {'OPR','SDR'}.issubset(df.columns):
        opr_sdr_ok = (df['OPR'].fillna(0) + df['SDR'].fillna(0)) > 0
    else:
        opr_sdr_ok = pd.Series(True, index=df.index)
    return (has_src == 1) & flags_ok & opr_sdr_ok

def fit_scaler_pca(X_fit: pd.DataFrame, mode: str,
                   n_components: int, var_target: float):
    """Fit StandardScaler and PCA on valid rows."""
    scaler = StandardScaler()
    Xz = scaler.fit_transform(X_fit)

    # Fit full PCA first to read explained variance profile
    pca_full = PCA().fit(Xz)
    evr = pca_full.explained_variance_ratio_
    cum = np.cumsum(evr)

    if mode == "var":
        k = int(np.searchsorted(cum, var_target) + 1)
    else:
        k = n_components

    # Refit PCA for just the chosen number of components
    pca = PCA(n_components=k).fit(Xz)
    return scaler, pca, evr, cum, k

def transform_all_rows(scaler: StandardScaler, pca: PCA,
                       X_all: pd.DataFrame) -> np.ndarray:
    Xz_all = scaler.transform(X_all)
    pcs_all = pca.transform(Xz_all)  # shape: (n_samples, k)
    return pcs_all

# --------------------------- CONTROL FLOW ------------------------------

# 1) Load and join
df = join_on_name(SUPERHEROES_INFO_POWERS_URL, SUPERHEROES_POWERS_URL)

# --- Tidy: drop redundant/merge-helper columns and dupes early ---
drop_exact = [c for c in ['hero_names', 'Unnamed: 0'] if c in df.columns]
df = df.drop(columns=drop_exact, errors='ignore')

# If merge created *_x/*_y duplicates, prefer the left side (_x) and drop _y
dupe_pairs = [(c, c.replace('_x', '_y')) for c in df.columns if c.endswith('_x')]
to_drop = [b for a, b in dupe_pairs if b in df.columns]
df = df.rename(columns={a: a[:-2] for a, b in dupe_pairs})  # strip _x
df = df.drop(columns=to_drop, errors='ignore')

# 2) Build power matrix X
X = build_power_matrix(df)

# 3) Compute validity mask (keep rows with real evidence)
mask = valid_power_mask(df, X)

# 4) Fit scaler + PCA on valid rows
X_fit = X.loc[mask].fillna(0).astype(float)
scaler, pca, evr, cum, k = fit_scaler_pca(
    X_fit, MODE, N_COMPONENTS, VAR_TARGET
)

print("Explained variance ratios (first 10):",
      [round(v, 3) for v in evr[:10]])
print("Cumulative (first 10):",
      [round(c, 3) for c in cum[:10]])
print(f"Chosen k={k}  (cumulative variance @k={cum[k-1]:.3f})")

# 5) Transform all rows, attach PCA only for valid ones
X_all = X.fillna(0).astype(float)
pcs_all = transform_all_rows(scaler, pca, X_all)  # (n_rows, k)

# Prepare PCA columns; invalid rows get NaN
for i in range(k):
    col = f'PCA_PC{i+1}'
    vals = pd.Series(np.nan, index=df.index)
    vals.loc[mask] = np.round(pcs_all[mask.values, i], 2)
    df[col] = vals

# Validity flag for downstream steps
df['PCA_valid'] = mask

# 6) Reorder columns so PCA block is on the right (no PCA_Power_Score)
pca_cols = [f'PCA_PC{i+1}' for i in range(k)]
tail_cols = ['PCA_valid'] + pca_cols
base_cols = [c for c in df.columns if c not in tail_cols]
df = df[base_cols + tail_cols]

# 7) Preview and report
print("\n=== PCA Summary Report ===")
print(f"Rows used for PCA fit: {int(mask.sum())}")
print(f"Rows excluded (insufficient power data): {int((~mask).sum())}")
print(f"Number of components retained: {k}")
print(f"Cumulative variance explained: {cum[k-1]:.3f} ({cum[k-1]*100:.1f}%)\n")

# Show first few principal component columns
preview_cols = ['name', 'Species', 'Alignment', 'PCA_valid'] + \
               [f'PCA_PC{i+1}' for i in range(min(5, k))]
print("Sample of first few principal components per hero (rows truncated):")
print(df[preview_cols].head())

# 8) Export version with PCA block at the end
print("\nBuilding 'superheroes_info_powers2.csv' with PCA columns appended...")
keep_info = ['name','Species','Gender','Alignment','Publisher',
             'Height','Weight','OPR','SDR','has_powers_source']
keep_info = [c for c in keep_info if c in df.columns]

# Save new INFO_POWERS dataset with PCA values
df_info_powers2 = df[keep_info + ['PCA_valid'] + pca_cols].copy()
df_info_powers2.to_csv(INFO_POWERS2_FILE, index=False)
print("File saved successfully.")

### Listing 3-4: K-Means Clustering on PCA Features
This code performs K-Means clustering on the PCA-transformed superhero power data. It uses a specified number of principal components to group superheroes into clusters. The code then reports the cluster sizes, silhouette score, and provides a basic analysis of cluster characteristics based on attributes like Species and Alignment. Finally, it visualizes the clusters using a scatter plot of the first two principal components.

In [ ]:
# K-Means clustering on PCA features with a basic report and plot.
# Set SUPERHEROES_INFO_POWERS2_URL to your CSV location before running.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# =========================
# Constants
# =========================
N_PCS_DEFAULT = 60   # number of PCA components to use by default
K_CLUSTERS    = 3    # number of clusters for K-Means
RAND_SEED     = 42

# =========================
# Helpers
# =========================
def _to_bool(x):
    if isinstance(x, str):
        return x.strip().lower() in ("true", "t", "1", "yes")
    try:
        return bool(int(x))
    except Exception:
        return bool(x)

def build_features(df, n_pcs=N_PCS_DEFAULT):
    """Return X and df_feat filtered for clustering on the first n_pcs PCs."""
    mask = pd.Series(True, index=df.index)
    if "PCA_valid" in df.columns:
        mask &= df["PCA_valid"].apply(_to_bool)
    if "has_powers_source" in df.columns:
        mask &= df["has_powers_source"].apply(_to_bool)
    df = df[mask].copy()

    # Allow up to the first 60 components if present, then slice to n_pcs
    pc_cols_all = [f"PCA_PC{i}" for i in range(1, 61)]
    pc_cols = [c for c in pc_cols_all if c in df.columns][:n_pcs]
    if not pc_cols:
        raise ValueError("No PCA_PC columns found in the dataset.")

    feats = df[pc_cols].astype(float).dropna()
    df_feat = df.loc[feats.index].copy()
    return feats.to_numpy(), df_feat, pc_cols

def basic_report(df_feat, labels, sil):
    df_feat = df_feat.copy()
    df_feat["Cluster"] = labels
    print(f"Chosen K: {len(np.unique(labels))}")
    print(f"Silhouette score: {sil:.3f}\n")

    if "Alignment" in df_feat.columns:
        print("Cluster vs Alignment (counts):")
        print(pd.crosstab(df_feat["Cluster"], df_feat["Alignment"]))
        print()

    if "Species" in df_feat.columns:
        print("Cluster vs Species (top 10):")
        species_ct = pd.crosstab(df_feat["Cluster"], df_feat["Species"])
        top = species_ct.sum().sort_values(ascending=False).head(10).index
        print(species_ct[top])
        print()

    print("Sample heroes per cluster:")
    if "name" in df_feat.columns:
        for c in sorted(df_feat["Cluster"].unique()):
            names = df_feat.loc[df_feat["Cluster"] == c, "name"].head(10)
            print(f"Cluster {c}: {', '.join(names)}")
    print()

def plot_pc_scatter(df_feat, labels, title):
    dfp = df_feat.copy()
    dfp["Cluster"] = labels
    colors = ListedColormap(["black", "red", "blue"])
    plt.figure(figsize=(8, 6))
    sc = plt.scatter(
        dfp["PCA_PC1"], dfp["PCA_PC2"],
        c=dfp["Cluster"], cmap=colors, s=40, alpha=0.6, edgecolor="none"
    )
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title(title)
    cbar = plt.colorbar(sc)
    cbar.set_label("Cluster")
    plt.tight_layout()
    plt.show()

def attach_kmeans_clusters(df: pd.DataFrame,
                           n_pcs: int = N_PCS_DEFAULT,
                           k: int = K_CLUSTERS,
                           label_col: str = "Cluster",
                           save_path: str | None = None,
                           random_state: int = RAND_SEED):
    """
    Fit K-Means on the first n_pcs PCA columns and add a 'Cluster' column.

    Returns (df_with_labels, kmeans_model, used_pc_columns, silhouette_score).

    Tip: If you later train a supervised model to predict these clusters,
    use a different feature set than these PCs to avoid leakage.
    """
    mask = pd.Series(True, index=df.index)
    if "PCA_valid" in df.columns:
        mask &= df["PCA_valid"].astype(str).str.lower().isin(["true", "t", "1", "yes"])
    if "has_powers_source" in df.columns:
        mask &= df["has_powers_source"].astype(str).str.lower().isin(["true", "t", "1", "yes"])

    pc_cols = [f"PCA_PC{i}" for i in range(1, 61)]
    pc_cols = [c for c in pc_cols if c in df.columns][:n_pcs]
    if not pc_cols:
        raise ValueError("No PCA_PC* columns found.")

    feats = df.loc[mask, pc_cols].apply(pd.to_numeric, errors="coerce").dropna()
    X = feats.to_numpy()

    km = KMeans(n_clusters=k, n_init=20, max_iter=300, random_state=random_state)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels) if len(np.unique(labels)) > 1 else float("nan")

    out = df.copy()
    out[label_col] = np.nan
    out.loc[feats.index, label_col] = labels

    if save_path:
        out.to_csv(save_path, index=False)
        print(f"Wrote: {save_path}")

    print(f"Rows labeled: {len(feats)} / {len(df)} | PCs used: {len(pc_cols)}")
    print(f"Chosen K: {k} | Silhouette: {sil:.3f}")
    return out, km, pc_cols, sil

# --------------------------- CONTROL FLOW ------------------------------
# Step 1 - Load the dataset from a CSV URL or local path.
df = pd.read_csv(SUPERHEROES_INFO_POWERS2_URL)

# Step 2 - Build the feature matrix from the first N_PCS_DEFAULT components.
#          PCA compresses 160+ power columns into fewer, denoised axes.
X, df_feat, feature_cols = build_features(df, n_pcs=N_PCS_DEFAULT)
print(f"Rows used: {X.shape[0]}   Features used: {X.shape[1]}")

# Step 3 - Fit K-Means with K=K_CLUSTERS.
#          Silhouette will help us judge separation quality later.
km = KMeans(
    n_clusters=K_CLUSTERS,   # number of clusters to find
    random_state=RAND_SEED,  # fixed seed for repeatable results
    n_init=20,               # try 20 centroid seeds, keep the best
    max_iter=300             # cap refinement steps
)
labels = km.fit_predict(X)   # run K-Means clustering

# Step 4 - Compute a quick quality score and print a basic report.
#          Silhouette ranges from -1 to 1. Higher means cleaner separation.
sil = silhouette_score(X, labels) if len(np.unique(labels)) > 1 else float("nan")
basic_report(df_feat, labels, sil)

# Step 5 - Plot clusters on PC1 vs PC2 for a visual read of the grouping.
plot_pc_scatter(
    df_feat,
    labels,
    f"K-Means on {N_PCS_DEFAULT} PCs (K={K_CLUSTERS})"
)

# Step 6 - (Optional) Write a new dataset with the added 'Cluster' column.
df3, km_model, used_pcs, sil = attach_kmeans_clusters(
    df,
    n_pcs=N_PCS_DEFAULT,     # keep in sync with the features above
    k=K_CLUSTERS,            # number of clusters
    label_col="Cluster",
    save_path=INFO_POWERS3_FILE,
    random_state=RAND_SEED
)

### Listing 3-4A: Analyzing K-Means Cluster Characteristics
This section examines the characteristics of the superhero clusters identified by K-Means in Listing 3-4. By running statistics across the groups, the code reveals notable similarities and trends within each cluster. This helps to understand the composition of the clusters based on attributes like Species, Alignment, and power levels, providing insights into the meaningfulness of the discovered groupings.

In [ ]:
# REQUIRES: df_feat, labels
# Cluster trend profiler: counts, top categories, numeric summaries.
# Call: profile_cluster_trends(df_feat, labels)

import numpy as np
import pandas as pd

def _pct_table(s, top_n=8):
    ct = s.value_counts(dropna=False)
    pct = (ct / ct.sum()).round(3)
    out = pd.concat([ct.rename("count"), pct.rename("pct")], axis=1)
    return out.head(top_n)

def profile_cluster_trends(df_feat, labels,
                           cat_cols=("Species", "Alignment", "Gender"),
                           num_cols=("OPR", "SDR", "Height", "Weight"),
                           top_n=8):
    fr = df_feat.copy()
    fr["Cluster"] = labels
    print("Cluster sizes:")
    print(fr["Cluster"].value_counts().sort_index(), "\n")

    # Categorical snapshots
    for col in cat_cols:
        if col not in fr.columns:
            continue
        print(f"Top {top_n} {col} values by cluster:")
        for c in sorted(fr["Cluster"].unique()):
            tab = _pct_table(fr.loc[fr["Cluster"] == c, col], top_n=top_n)
            print(f"\nCluster {c} — {col}")
            print(tab)
        print()

    # Numeric summaries
    keep = [c for c in num_cols if c in fr.columns]
    if keep:
        print("Numeric summaries by cluster:")
        summ = (fr.groupby("Cluster")[keep]
                  .agg(["mean", "median", "std", "min", "max", "count"]))
        print(summ.round(2), "\n")

    # Simple power tier from OPR and SDR if present
    if {"OPR", "SDR"}.issubset(fr.columns):
        z = fr[["OPR", "SDR"]].apply(
            lambda c: (c - c.mean()) / c.std(ddof=0)
        )
        fr["power_index"] = z.sum(axis=1)
        tier_mean = (fr.groupby("Cluster")["power_index"]
                       .mean().sort_values())
        order = tier_mean.index.tolist()
        names = {order[0]: "Low", order[-1]: "High"}
        if len(order) >= 3:
            names[order[len(order)//2]] = "Medium"
        print("Average power_index by cluster:")
        print(tier_mean.round(3))
        print("Assigned tiers:", names, "\n")

# Example call right after clustering:
profile_cluster_trends(df_feat, labels)

### Listing 3-5: Cosine Similarity Calculation for Superheroes Relationships
Calculates the cosine similarity between superheroes, identifying the most and least similar pairs based on their powers. The results highlight relationships between characters.

In [ ]:
# REQUIRES: Assumes df_feat (with PCA_PC* columns) and labels are in memory.

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# Helpers (keep in Colab)
# =========================
def _pca_feature_matrix(df_feat, n_pcs=None):
    """Build a numeric feature matrix from PCA_PC* columns."""
    pc_cols = [c for c in df_feat.columns if c.startswith("PCA_PC")]
    pc_cols = sorted(pc_cols, key=lambda c: int(c.replace("PCA_PC", "")))
    if n_pcs is not None:
        pc_cols = pc_cols[:n_pcs]
    if not pc_cols:
        raise ValueError("No PCA_PC* columns found in df_feat.")
    X = df_feat[pc_cols].astype(float).to_numpy()
    return X, pc_cols

def _upper_triangle_pairs(n):
    """Indices for unique unordered pairs (i < j)."""
    return np.triu_indices(n, k=1)

def _per_cluster_within_means(S, labels):
    """Mean within-cluster cosine per cluster."""
    out = []
    labs = np.asarray(labels)
    for c in sorted(np.unique(labs)):
        idx = np.where(labs == c)[0]
        if len(idx) < 2:
            out.append((c, np.nan, 0))
            continue
        Sc = S[np.ix_(idx, idx)]
        iu = np.triu_indices(len(idx), 1)
        vals = Sc[iu]
        out.append((c, float(np.mean(vals)), int(vals.size)))
    return out

def _top_within_pairs(S, labels, names=None, top_k=5):
    """Top similar pairs inside each cluster."""
    labs = np.asarray(labels)
    rows = []
    for c in sorted(np.unique(labs)):
        idx = np.where(labs == c)[0]
        if len(idx) < 2:
            rows.append((c, []))
            continue
        Sc = S[np.ix_(idx, idx)]
        iu = np.triu_indices(len(idx), 1)
        vals = Sc[iu]
        order = np.argsort(-vals)[:top_k]
        pairs = []
        for r in order:
            i = idx[iu[0][r]]
            j = idx[iu[1][r]]
            a = names[i] if names is not None else str(i)
            b = names[j] if names is not None else str(j)
            pairs.append((a, b, float(vals[r])))
        rows.append((c, pairs))
    return rows

# =========================
# Main flow (show in book)
# =========================

# Step 1 - Build the feature matrix used for cosine.
#          We use the same PCA axes we used for clustering.
X_cos, used_cols = _pca_feature_matrix(df_feat, n_pcs=20)
print(f"Cosine features: {len(used_cols)} columns "
      f"({used_cols[0]}, {used_cols[1]}, ...)")

# Step 2 - Compute cosine similarity between all heroes.
#          Values range from -1 (opposites) to 1 (identical direction).
S = cosine_similarity(X_cos)

# Step 3 - Compare similarity within clusters vs between clusters.
labs = np.asarray(labels)          # cluster label for each hero as a NumPy array
n = len(labs)                      # number of heroes
iu = _upper_triangle_pairs(n)      # all unique hero pairs (i < j)
same = labs[iu[0]] == labs[iu[1]]  # True if a pair is in the same cluster
within_vals = S[iu][same]          # cosine scores for same-cluster pairs
between_vals = S[iu][~same]        # cosine scores for cross-cluster pairs

print("Cosine similarity (overall):")
print(f"  Within clusters:  mean={within_vals.mean():.3f}  "
      f"pairs={within_vals.size}")
print(f"  Between clusters: mean={between_vals.mean():.3f} "
      f"pairs={between_vals.size}\n")

# Step 4 - Show which clusters are tight vs diffuse.
print("Per-cluster within means:")
for c, m, cnt in _per_cluster_within_means(S, labs):
    print(f"  Cluster {c}: mean={m:.3f} pairs={cnt}")
print()

# Step 5 - List a few most-similar hero pairs inside each cluster.
names = df_feat["name"].tolist() if "name" in df_feat.columns else None
print("Top within-cluster pairs:")
for c, pairs in _top_within_pairs(S, labs, names=names, top_k=5):
    if not pairs:
        print(f"  Cluster {c}: not enough members")
        continue
    print(f"  Cluster {c}:")
    for a, b, v in pairs:
        print(f"    {a}  ~  {b}   cos={v:.3f}")

### Listing 3-6A: Target screening for fine-tuning candidates

In [ ]:
# REQUIRES: SUPERHEROES_INFO_POWERS2_URL
# Target screening for fine-tuning candidates

import numpy as np
import pandas as pd

# ---------- Helpers ----------

def make_power_tier(df: pd.DataFrame) -> pd.Series:
    """Low/Medium/High from z(OPR)+z(SDR); preserves NaNs."""
    if {"OPR", "SDR"}.issubset(df.columns):
        s = df[["OPR", "SDR"]].apply(pd.to_numeric, errors="coerce")
        z = (s - s.mean()) / s.std(ddof=0)
        idx = z.sum(axis=1)
        q1, q2 = idx.quantile([1/3, 2/3])
        bins = [-np.inf, q1, q2, np.inf]
        out = pd.cut(idx, bins=bins, labels=["Low", "Medium", "High"])
        out.name = "Power_Tier"
        return out
    return pd.Series(pd.NA, index=df.index, name="Power_Tier", dtype="object")

def make_publisher3(df: pd.DataFrame) -> pd.Series:
    """Collapse Publisher → Marvel / DC / Other."""
    pub = df.get("Publisher")
    if pub is None:
        return pd.Series(pd.NA, index=df.index, name="Publisher3", dtype="object")
    s = pub.astype(str)
    is_marvel = s.str.contains("marvel", case=False, na=False)
    is_dc = s.str.contains("dc", case=False, na=False)
    out = np.where(is_marvel, "Marvel", np.where(is_dc, "DC", "Other"))
    return pd.Series(out, index=df.index, name="Publisher3")

def target_health(df: pd.DataFrame, col: str) -> dict:
    """Quick diagnostics for a classification target."""
    s = df[col]
    miss = float(s.isna().mean())
    k = int(s.dropna().nunique())
    vc_all = s.value_counts(dropna=False).sort_values(ascending=False)
    vc_nomiss = s.dropna().value_counts()
    maj = float(vc_nomiss.iloc[0] / vc_nomiss.sum()) if not vc_nomiss.empty else np.nan
    if np.isnan(maj):
        verdict = "unusable (no data)"
    elif maj >= 0.80:
        verdict = "very imbalanced"
    elif maj >= 0.60:
        verdict = "imbalanced but workable"
    else:
        verdict = "reasonably balanced"
    return {
        "target": col,
        "classes": k,
        "missing_pct": round(miss, 3),
        "majority_baseline": round(maj, 3) if not np.isnan(maj) else np.nan,
        "verdict": verdict,
        "counts": vc_all.to_dict(),
    }

def print_health_summary(results: list):
    """Pretty print a compact summary for each target."""
    for r in results:
        print(f"\n=== {r['target']} ===")
        print(f"classes: {r['classes']} | missing: {r['missing_pct']}")
        print(f"majority baseline: {r['majority_baseline']} | {r['verdict']}")
        counts = pd.Series(r["counts"]).sort_values(ascending=False).head(8)
        print(counts.to_string())

# ---------- Main ----------

df = pd.read_csv(SUPERHEROES_INFO_POWERS2_URL).copy()

# Derived targets
df["Power_Tier"] = make_power_tier(df)
df["Publisher3"] = make_publisher3(df)

# Choose targets present in the file
candidates = [
    c for c in ["Alignment", "Species", "Gender",
                "has_powers_source", "Power_Tier", "Publisher3"]
    if c in df.columns
]

results = [target_health(df, c) for c in candidates]
print_health_summary(results)

### Listing 3-6: (Tuning Step 1) Build Target, Select Features and Encode
This listing prepares the dataset for supervised modeling. It starts by loading the updated superheroes_info_powers2 file and creating a clean binary target (“Marvel” vs “DC”) with make_target. Next, it organizes the feature set in one place using select_columns, keeping 40 PCA components, a handful of numeric fields (Height, Weight, OPR, SDR), and text fields (Alignment, Species, Gender)—intentionally excluding name. With the features set, we encode the categorical variables and target using scikit-learn’s LabelEncoder, producing numeric inputs ready for training. Finally, we print quick stats so you can check class balance and feature shapes before moving ahead.

In [ ]:
# REQUIRES: SUPERHEROES_INFO_POWERS2_URL
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder


# Helpers (omitted from the printed book)

def make_target(df, source_col, keep_values, mapping):
    """
    Filter to rows whose publisher matches any value in keep_values (case-insens).
    Then map those rows to a clean target label using 'mapping'.
    """
    s = df[source_col].astype(str)

    # Build a safe OR-pattern (escape in case values contain special chars)
    or_pat = "|".join(re.escape(v) for v in keep_values)

    mask = s.str.contains(or_pat, case=False, na=False)
    out = df[mask].copy()

    # Row-wise mapping to "Marvel"/"DC" from the raw publisher string
    out["target"] = np.select(
        [out[source_col].str.contains(k, case=False, na=False) for k in mapping],
        [mapping[k] for k in mapping],
        default="Other"  # should not occur because of mask, but kept for safety
    )
    return out


def select_columns(df, base_num_cols, text_cols, n_pcs=40):
    """
    Pick features:
      - numeric: PCA_PC1..PCA_PC{n_pcs} + base numeric columns
      - text: the provided text columns
    Require full PCA block; coerce text to strings to avoid NaNs.
    """
    pc_cols = [f"PCA_PC{i}" for i in range(1, n_pcs + 1)]
    df = df.dropna(subset=pc_cols).copy()

    num_cols = pc_cols + [c for c in base_num_cols if c in df.columns]
    txt_cols = [c for c in text_cols if c in df.columns]

    for c in txt_cols:
        df[c] = df[c].astype(str).fillna("NA")

    return df, num_cols, txt_cols

# ---------- control flow ----------

# Load latest version of the superheroes dataset
df = pd.read_csv(SUPERHEROES_INFO_POWERS2_URL).copy()

target_feature = "Publisher"              # column predict

# Target: Marvel vs DC (concert from full strings e.g., 'Marvel Comics')
df = make_target(
    df,
    source_col=target_feature,
    keep_values=["marvel comics", "dc comics"],
    mapping={"marvel comics": "Marvel", "dc comics": "DC"}
)

# Column picks include both Numeric and Textual Fields
base_numeric = ["Height", "Weight", "OPR", "SDR"]
text_fields  = ["Alignment", "Species", "Gender"]  # do NOT include 'name'

# Select features in one place (adds PCA_PC1..PCA_PC40 to numeric_cols)
df, numeric_cols, text_cols = select_columns(
    df, base_num_cols=base_numeric, text_cols=text_fields, n_pcs=40
)

# Encode Features and Target
X = df[text_cols + numeric_cols]          # feature frame
le = LabelEncoder()                       # map labels to 0..K-1
y = le.fit_transform(df["target"])        # encodes "Marvel"/"DC" -> ints

# Print Statistics
print(f"Rows with full PC block: {len(df)}")
print("Target counts:", df["target"].value_counts().to_dict())
print(f"Text cols ({len(text_cols)}): {text_cols}")
print(f"Numeric cols ({len(numeric_cols)}): {numeric_cols}...")

### Listing 3-7: (Tuning Step 2) Preparing Train/Test and Establishing a Baseline
This code sets up the superhero dataset for training and evaluation. It splits the data into train and test sets while preserving the Marvel/DC class balance, then applies preprocessing: one-hot encoding for text fields (alignment, species, gender) and median imputation for numeric fields (Height, Weight, OPR, SDR, plus PCA features). With the features transformed, it establishes a baseline by always predicting the majority class from the training labels, giving a simple reference point for accuracy and macro-F1 before moving on to more advanced models.

In [ ]:
# REQUIRES: X, y, text_cols, numeric_cols
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score

# Helpers (omitted from the printed book)
def build_preprocessor(text_cols, num_cols):
    """One-hot for text; median impute for numeric."""
    return ColumnTransformer(
        transformers=[
            ("text",
             OneHotEncoder(handle_unknown="ignore", sparse_output=True),
             text_cols),
            ("num",
             Pipeline([("imputer", SimpleImputer(strategy="median"))]),
             num_cols),
        ],
        remainder="drop",
    )

def report(tag, y_true, y_pred):
    """Print accuracy and macro-F1 in one line."""
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    print(f"{tag:24} acc: {acc:.3f} | macro-F1: {f1m:.3f}")
    return acc, f1m

# ----------- CONTROL FLOW -----------

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# 1. Train/test split (80/20) with stratify to preserve class ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# 2. Preprocessing: One-hot encode text, median impute numeric
preprocessor = ColumnTransformer(
    transformers=[
        ("text",
         OneHotEncoder(handle_unknown="ignore", sparse_output=True),
         text_cols),
        ("num",
         Pipeline([("imputer", SimpleImputer(strategy="median"))]),
         numeric_cols),
    ],
    remainder="drop",
)

# 3. Fit on training data only (avoid leakage)
X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded  = preprocessor.transform(X_test)

# 4. Baseline: always predict the majority class from training labels
majority_class = np.bincount(y_train).argmax()
y_base = np.full_like(y_test, majority_class)

# 5. Calculate and print base line accuracy score
acc = accuracy_score(y_test, y_base)
f1m = f1_score(y_test, y_base, average="macro")

print(f"Train size: {X_train.shape[0]} | Test size: {X_test.shape[0]}")
print("X_train_encoded:", X_train_encoded.shape,
      "| X_test_encoded:", X_test_encoded.shape)
print("y_train:", y_train.size, "| y_test:", y_test.size)
print(f"Baseline (majority) acc: {acc:.3f} | macro-F1: {f1m:.3f}")

### Listing 3-8: (Tuning Step 3) Pre-Fine-Tune GradientBoosting Classifier

This snippet trains a vanilla HistGradientBoostingClassifier on the already-encoded features to establish a baseline we’ll try to beat in the next step. It converts the sparse design matrices to dense (HGB requires dense), fits the model, and reports accuracy and macro-F1 plus a class-wise report.
Prereqs from the previous cell: X_train_encoded, X_test_encoded, y_train, y_test, and le (a LabelEncoder fitted on the target) are already defined.

In [ ]:
# REQUIRES: X_train_encoded, X_test_encoded, y_train, y_test, le
# Pre-fine-tune model: HistGradientBoosting (simple baseline)
# Goal: a clean starting point we can beat in the next step

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# HGB needs dense arrays (not sparse CSR). Convert once.
X_train_dense = (X_train_encoded.toarray()
                 if hasattr(X_train_encoded, "toarray")
                 else X_train_encoded)
X_test_dense = (X_test_encoded.toarray()
                if hasattr(X_test_encoded, "toarray")
                else X_test_encoded)

# Barebones classifier
hgb = HistGradientBoostingClassifier()

# Fit → Predict → Score
hgb.fit(X_train_dense, y_train)   # train the model
y_pred = hgb.predict(X_test_dense)

# Prepare and Print Classification Report
acc = accuracy_score(y_test, y_pred)
f1m = f1_score(y_test, y_pred, average="macro")

print(f"HGB (baseline)   acc: {acc:.3f} | macro-F1: {f1m:.3f}")
print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

### Listing 3-9: (Tuning Step 4) Fine Tuning of GradientBoosting Classifier
We now apply the **HistGradientBoostingClassifier** with parameters tuned through earlier experiments. The learning rate, tree depth, and L2 regularization were adjusted to strike a balance between model flexibility and generalization. This “best pass” run shows how testing a range of settings lets us converge on a strong configuration. The final model combines encoded text and numeric features, demonstrating how systematic fine-tuning can push accuracy and macro-F1 well above  
baseline performance.

In [ ]:
# REQUIRES: X_train_dense, X_test_dense, y_test, y_train, le
# Fine-tuned HistGradientBoosting

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

clf = HistGradientBoostingClassifier(
    learning_rate=0.3,        # step size for boosting updates
    max_depth=5,              # depth of each tree (controls complexity)
    l2_regularization=0.1,    # weight decay to reduce overfitting
    max_iter=100,             # enough trees to converge
    random_state=42           # reproducible results
)

# Fit the model to the training data
clf.fit(X_train_dense, y_train)   # train with encoded features
pred = clf.predict(X_test_dense)  # predict on held-out test set

# Prepare and Print Classification Report
# Evaluate results with accuracy and macro-F1
acc = accuracy_score(y_test, pred)
f1m = f1_score(y_test, pred, average="macro")

print(f"HGB (best params)  acc: {acc:.3f} | macro-F1: {f1m:.3f}\n")
print("Detailed report:")
print(classification_report(y_test, pred, target_names=le.classes_))

### Plot history of tunning accuracy

In [ ]:
# REQUIRES: labels, values
import matplotlib.pyplot as plt

labels = ["3-class\nbaseline", "Binary\nbaseline", "Untuned\nHGB", "Tuned\nHGB"]
values = [52, 65, 77, 84]
colors = ["yellow", "gray", "red", "blue"]

plt.figure(figsize=(6, 4))
bars = plt.bar(labels, values, color=colors)
plt.title("Model Progress: Accuracy (%)")
plt.ylabel("Accuracy (%)")
plt.ylim(0, 100)

for bar, val in zip(bars, values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 1, f"{val}%",
             ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

### Listing 3-9A: Model Selection and Fine-Tuning Analysis Sweep

This program runs three strong classifiers side by side: LinearSVC (linear support vector machine), Logistic Regression, and HistGradientBoosting. Each is trained on the same encoded dataset, with small sweeps over their key parameters (regularization strength for linear models, learning rate and tree depth for boosting). The results show not only which model performs best, but also which parameter settings give the highest accuracy and macro-F1, guiding us toward a tuned final model.

In [ ]:
# REQUIRES: X_train_encoded, X_test_encoded, X_train_dense, X_test_dense,
#          y_train, y_test

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# ---------- helpers ----------
def evaluate(tag, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    print(f"{tag:28} acc: {acc:.3f} | macro-F1: {f1m:.3f}")
    return acc, f1m, y_pred

def best_of(rows):
    # rows: [(acc, f1m, pred, params_dict), ...]
    return max(rows, key=lambda t: (t[0], t[1]))

# ---------- LinearSVC sweep (sparse OK) ----------
svc_trials = []
for C in [0.5, 1.0, 1.5, 2.0]:
    for balanced in (True, False):
        clf = LinearSVC(
            C=C,
            class_weight=("balanced" if balanced else None),
            random_state=42
        )
        clf.fit(X_train_encoded, y_train)
        pred = clf.predict(X_test_encoded)
        svc_trials.append(
            evaluate(f"SVC C={C:<3} bal={balanced}", y_test, pred) +
            ({"C": C, "balanced": balanced},)
        )

svc_best = best_of(svc_trials)
print("\nBest LinearSVC params:", svc_best[3])
print(classification_report(y_test, svc_best[2]))

# ---------- Logistic Regression sweep (sparse OK) ----------
lr_trials = []
for C in [0.5, 1.0, 2.0, 4.0]:
    for balanced in (True, False):
        clf = LogisticRegression(
            C=C,
            penalty="l2",
            solver="liblinear",          # solid with sparse & binary
            class_weight=("balanced" if balanced else None),
            max_iter=2000,
            n_jobs=-1,
            random_state=42
        )
        clf.fit(X_train_encoded, y_train)
        pred = clf.predict(X_test_encoded)
        lr_trials.append(
            evaluate(f"LogReg C={C:<3} bal={balanced}", y_test, pred) +
            ({"C": C, "balanced": balanced},)
        )

lr_best = best_of(lr_trials)
print("\nBest LogisticRegression params:", lr_best[3])
print(classification_report(y_test, lr_best[2]))

# ---------- HistGradientBoosting sweep (dense required) ----------
# Include the known strong combo: lr=0.3, depth=5, l2=0.1
hgb_trials = []
for lr in [0.10, 0.20, 0.30, 0.40]:
    for depth in [5]:
        for l2 in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
            clf = HistGradientBoostingClassifier(
                learning_rate=lr,
                max_depth=depth,
                l2_regularization=l2,
                random_state=42
            )
            clf.fit(X_train_dense, y_train)
            pred = clf.predict(X_test_dense)
            hgb_trials.append(
                evaluate(f"HGB lr={lr:<3} d={depth} l2={l2:<3}", y_test, pred) +
                ({"lr": lr, "depth": depth, "l2": l2},)
            )

hgb_best = best_of(hgb_trials)
print("\nBest HistGB params:", hgb_best[3])
print(classification_report(y_test, hgb_best[2]))

# ---------- Overall winner ----------
winners = [
    ("LinearSVC",) + svc_best,   # (name, acc, f1, pred, params)
    ("LogReg",)    + lr_best,
    ("HistGB",)    + hgb_best,
]
overall = max(winners, key=lambda t: (t[1], t[2]))

print("\n=== Overall best ===")
print("Model:", overall[0], "| params:", overall[4])
print(f"acc: {overall[1]:.3f} | macro-F1: {overall[2]:.3f}\n")
print("Detailed report (overall best):")
print(classification_report(y_test, overall[3]))

### Listing 3-9B - SMOTE sniff test

In [ ]:
# REQUIRES: X_train_encoded, X_test_encoded, y_train, y_test from earlier cells

from collections import Counter
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score

def to_dense(X):
    return X.toarray() if hasattr(X, "toarray") else X

def score(tag, clf, Xtr, ytr, Xte, yte):
    clf.fit(Xtr, ytr)
    pred = clf.predict(Xte)
    acc = accuracy_score(yte, pred)
    f1m = f1_score(yte, pred, average="macro")
    print(f"{tag:22} acc: {acc:.3f} | macro-F1: {f1m:.3f}")
    return acc, f1m

# Convert to dense (HGB + SMOTE need dense arrays)
Xtr = to_dense(X_train_encoded)
Xte = to_dense(X_test_encoded)

# Tuned HGB settings from our fine-tuning
hgb = HistGradientBoostingClassifier(
    learning_rate=0.3, max_depth=5, l2_regularization=0.1,
    max_iter=100, random_state=42
)

# 1) No SMOTE
score("HGB (no SMOTE)", hgb, Xtr, y_train, Xte, y_test)

# 2) With SMOTE on the training set only
sm = SMOTE(random_state=42)
Xtr_sm, ytr_sm = sm.fit_resample(Xtr, y_train)

print("Train balance before:", Counter(y_train))
print("Train balance after :", Counter(ytr_sm))

score("HGB (with SMOTE)", hgb, Xtr_sm, ytr_sm, Xte, y_test)

### Listing 3-10: (Tuning Step 5) Visualizing a Confusion Matrix for Model Evaluation
Generates and visualizes the confusion matrix, showing the breakdown of correct and incorrect predictions for each class in the dataset.

In [ ]:
# Confusion Matrix for HistGradientBoosting predictions
# REQUIRES: y_test (true), pred (preds), and `le` from the target encoding step

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 1) Build the 2x2 confusion matrix in the encoder's class order
class_indices = np.arange(len(le.classes_))   # e.g., [0, 1]
cm = confusion_matrix(y_test, pred, labels=class_indices)
tick_names = le.classes_                      # e.g., ["DC Comics", "Marvel Comics"]

# 2) Create a cell-by-cell color grid:
#    [ [UL (TP class 0)=RED,  UR (FN class 0)=YELLOW],
#      [LL (FP class 0)=YELLOW, LR (TP class 1)=BLUE] ]
colors = np.array([
    [(1.0, 0.0, 0.0, 0.70),  (1.0, 1.0, 0.0, 0.70)],
    [(1.0, 1.0, 0.0, 0.70),  (0.0, 0.0, 1.0, 0.70)],
])

# 3) Plot the colored grid and overlay the counts
fig, ax = plt.subplots(figsize=(6, 5))
ax.imshow(colors, interpolation='none', aspect='equal')

# Add the numbers (bigger font)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, str(cm[i, j]),
                ha='center', va='center',
                fontsize=16, fontweight='bold', color='black')

# 4) Axes, ticks, and labels
ax.set_xticks([0, 1], labels=tick_names)
ax.set_yticks([0, 1], labels=tick_names)
ax.set_xlabel("Predicted", fontsize=12)
ax.set_ylabel("True", fontsize=12)
ax.set_title("Confusion Matrix (HistGB)", fontsize=13, pad=10)

# Optional: add thin gridlines between cells for readability
ax.set_xticks(np.arange(-0.5, 2, 1), minor=True)
ax.set_yticks(np.arange(-0.5, 2, 1), minor=True)
ax.grid(which='minor', color='white', linewidth=1.5)
ax.tick_params(which='minor', bottom=False, left=False)

plt.tight_layout()
plt.show()

### Listing 3-11 - SHAP values for HistGradientBoosting
Calculates SHAP (SHapley Additive exPlanations) values for the trained `HistGradientBoostingClassifier`. SHAP values help interpret the model's predictions by showing how much each feature contributes to the output for individual instances. The code ranks features by their average absolute SHAP value to identify the most important features in the model's predictions.

In [ ]:
# SHAP on HistGradientBoosting (numbers only)
# Requires: clf (fitted HGB), preprocessor, text_cols, numeric_cols,
#           X_train_encoded, X_test_encoded

import numpy as np
import shap

def to_dense(X):
    return X.toarray() if hasattr(X, "toarray") else X

# Dense matrices for HGB
X_train_dense = to_dense(X_train_encoded)
X_test_dense  = to_dense(X_test_encoded)

# Feature names from the ColumnTransformer (OneHot + numeric)
feat_names = preprocessor.get_feature_names_out(text_cols + numeric_cols)

# Use TreeExplainer with default model_output="raw" (required for HGB)
explainer = shap.TreeExplainer(clf)

# SHAP values for the test set; disable additivity check to avoid numerical issues
shap_vals = explainer.shap_values(X_test_dense, check_additivity=False)

# For binary classifiers, SHAP may return a list (old) or an array (new). Normalize:
if isinstance(shap_vals, list):
    shap_raw = shap_vals[1]        # positive class
else:
    shap_raw = shap_vals           # shape: (n_samples, n_features)

# Rank features by mean |SHAP|
mean_abs = np.mean(np.abs(shap_raw), axis=0)
order = np.argsort(-mean_abs)

top_k = 15
print("Top features by mean |SHAP| (HGB, raw margin):")
for rank, idx in enumerate(order[:top_k], 1):
    print(f"{rank:2d}. {feat_names[idx]}: {mean_abs[idx]:.5f}")

# Optional: show base value (raw margin) and a per-sample check
expected = float(np.atleast_1d(explainer.expected_value)[-1])  # scalar
row = 0
margin = expected + shap_raw[row].sum()  # raw margin (log-odds)
prob = 1.0 / (1.0 + np.exp(-margin))     # implied probability

print(f"\nBase value (raw margin): {expected:.5f}")
print(f"Sample {row} raw margin (expected + sum SHAP): {margin:.5f}")
print(f"Sample {row} implied probability (sigmoid(raw)): {prob:.3f}")

### Listing 3-12 – Beethoven Edge Feature Visualization

This program demonstrates how classical computer vision techniques extract visual features from images. It uses edge detection and gradient orientation analysis to highlight prominent contours in red, blue, and yellow—mimicking how early machine learning systems learned to perceive structure before the era of deep learning.

In [ ]:
# Beethoven Feature Visualization — Bold Red/Blue/Yellow Edges on White
# Downloads the public-domain portrait and renders classical CV features.

import urllib.request, numpy as np, cv2
from PIL import Image
import matplotlib.pyplot as plt

# ========= Download image =========
URL = "https://opensourceai-book.github.io/code/media/beethoven.png"
SAVE_AS = "Beethoven.jpg"
urllib.request.urlretrieve(URL, SAVE_AS)

# ========= Inputs =========
FILENAME = SAVE_AS                # downloaded file
MAX_SIDE = 1200                   # resize long side to speed up processing

# ========= One-knob sensitivity (0..1) =========
EDGE_SENSITIVITY = 0.75           # higher = keep more edges; lower = sparser

# ========= Derived settings (adjusted by sensitivity) =========
CANNY_LOW  = int(60  - 30*EDGE_SENSITIVITY)   # lower edge threshold
CANNY_HIGH = int(150 - 40*EDGE_SENSITIVITY)   # upper edge threshold
KEEP_TOP_PERCENT = 0.45 + 0.45*EDGE_SENSITIVITY  # fraction of strongest edges to keep
DILATE_ITERS = 2 if EDGE_SENSITIVITY < 0.8 else 3  # stroke thickness
MIN_REGION_SIZE = int(80 - 60*EDGE_SENSITIVITY)    # remove tiny blobs below this area
SMOOTH_MASK = 5                                    # blur kernel merging nearby fragments
POST_BLUR_KEEP = 0.20                              # post-blur threshold; lower keeps more
ALPHA_MIN, ALPHA_MAX = 0.90, 1.00                  # opacity range for strokes
UNDERLAY_GRAY = 200                                # light gray halo under strokes (0..255)

# ========= Superhero palette (RGB) =========
RED  = np.array([220,  35,  40], dtype=np.float32) # steep (near-vertical) edges
BLUE = np.array([ 50,  90, 210], dtype=np.float32) # broad midrange orientations
YEL  = np.array([255, 235,  80], dtype=np.float32) # shallow (near-horizontal) edges

# ---------- Load & resize ----------
img = Image.open(FILENAME).convert("RGB")
w, h = img.size
s = min(1.0, MAX_SIDE / max(w, h))
if s < 1.0:
    img = img.resize((int(w*s), int(h*s)), Image.Resampling.LANCZOS)
rgb = np.array(img, dtype=np.uint8)
H, W = rgb.shape[:2]

# ---------- Edge feature extraction ----------
gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
gray_blur = cv2.GaussianBlur(gray, (3,3), 0)

edges = cv2.Canny(gray_blur, CANNY_LOW, CANNY_HIGH)
edges = cv2.dilate(edges, np.ones((2,2), np.uint8), iterations=DILATE_ITERS)

gx = cv2.Sobel(gray_blur, cv2.CV_32F, 1, 0, ksize=3)
gy = cv2.Sobel(gray_blur, cv2.CV_32F, 0, 1, ksize=3)
mag = np.sqrt(gx*gx + gy*gy)
ang = (np.degrees(np.arctan2(gy, gx)) + 180.0) % 180.0

# ---------- Keep strongest edges ----------
mask_edges = edges > 0
m_vals = mag[mask_edges]
thr = np.quantile(m_vals, 1.0 - KEEP_TOP_PERCENT) if m_vals.size else 0.0
key = (mask_edges & (mag >= thr)).astype(np.uint8)

# Remove tiny blobs
n, labels_cc, stats, _ = cv2.connectedComponentsWithStats(key, connectivity=8)
for i in range(1, n):
    if stats[i, cv2.CC_STAT_AREA] < max(1, MIN_REGION_SIZE):
        key[labels_cc == i] = 0

# Merge fragments then re-threshold
key = cv2.GaussianBlur(key.astype(np.float32), (SMOOTH_MASK, SMOOTH_MASK), 0)
key = (key > POST_BLUR_KEEP).astype(np.uint8)

# Opacity from normalized magnitude
alpha = np.zeros_like(mag, dtype=np.float32)
if m_vals.size:
    mn, mx = m_vals.min(), m_vals.max()
    sel = key.astype(bool)
    alpha[sel] = (mag[sel] - mn) / (mx - mn + 1e-8)
alpha = (ALPHA_MIN + (ALPHA_MAX - ALPHA_MIN) * alpha).astype(np.float32)

# ---------- Color by orientation ----------
bin_yel  = (key==1) & (ang < 45)
bin_blue = (key==1) & (ang >= 45) & (ang < 135)
bin_red  = (key==1) & (ang >= 135)

# ---------- Compose on white ----------
canvas = np.ones((H, W, 3), dtype=np.uint8) * 255
if UNDERLAY_GRAY:
    glow = cv2.GaussianBlur(key.astype(np.float32), (5,5), 0)
    under = np.full((H, W, 3), UNDERLAY_GRAY, dtype=np.uint8)
    canvas = (canvas*(1.0 - glow[...,None]) + under*glow[...,None]).astype(np.uint8)

def paint(mask, color):
    a = alpha[..., None]
    overlay = (255.0*(1.0 - a) + color*a).astype(np.uint8)
    canvas[mask] = overlay[mask]

paint(bin_red,  RED)
paint(bin_blue, BLUE)
paint(bin_yel,  YEL)

# ---------- Save ----------
out_name = "beethoven_edges_on_white.png"
Image.fromarray(canvas).save(out_name)

plt.imshow(canvas); plt.axis("off"); plt.tight_layout(); plt.show()
print(f"Saved: {out_name}")